In [1]:
from ckanapi import RemoteCKAN
import time
import sqlite3
import pandas as pd

conn = sqlite3.connect('~/scidm')
c = conn.cursor()

In [2]:
info = {
    'lifesafety': 'E00', #生活安全及品質
    'publicinformation': 'I00', #公共資訊
    'travel': '900', #休閒旅遊
    'investment': '800', #投資理財
    'traffic': 'A00', #交通及通訊
    'business': '600', #開創事業
    'job': '500', #求職及就業
    'medical': 'B00', #就醫
    'study': '300', #求學及進修
    'election': 'D00', #選舉及投票
    'birth': '200', #出生及收養
    'elderlycare': 'G00', #老年安養
    'fertility': '100', #生育保健
    'lifeetiquette': 'H00', #生命禮儀
    'housepurchase': 'C00', #購屋及遷徙
    'army': '400', #服兵役
    'marriage': '700', #婚姻
    'retirement':'F00' #退休
}

In [49]:
scidm = RemoteCKAN('https://~', apikey='key')
packages = scidm.action.package_list()

#叫出已存在 sqlite 中跑過的 id
finish_id = pd.read_sql_query('select * from patchDone', conn)
finish_id_list = finish_id['id'].tolist()

#比對出未過跑準備要下去跑的存為 list
not_finish_packages = list(set(packages).symmetric_difference(set(finish_id_list)))

In [50]:
scidm_num = 0
other_data_num = 0

In [51]:
for package in not_finish_packages:
    now_time = time.strftime('%H:%M:%S',time.localtime())
    gov_data = scidm.action.package_show(id = package)
    
    #遇到 organization:null 的狀況時例外處理
    try:
        #確認是 ipgod 組織的資料
        if '政府開放資料彙整平台' in gov_data['organization']['title']:

            #確認 groups 還未更動過，如果 groups 為空 list，繼續以下動作
            if not gov_data['groups']:
                scidm_num += 1

                #從 extras 中取出所有的 key, value 檢查
                for key in gov_data['extras']:
                    #找到 key 是 categoryCode，和上方 info 字典比對
                    if 'categoryCode' in key['key']:
                        for infokey, infovalue in info.items():
                            if key['value'] == infovalue:
                                print(str(scidm_num)+'.',package, key['value'], infokey, now_time)
                                scidm.action.package_patch(id = package, groups = [{'name':infokey}])

                                #順利更新群組的資料將 id 加入 sqlite 中
                                c.execute('insert into patchDone(id) values(?)', (package,))
                                conn.commit()
                                time.sleep(0.5)

            else:
                #先前處理過的資料將 id 加入 sqlite 中
                c.execute('insert into patchDone(id) values(?)', (package,))
                conn.commit()
                scidm_num += 1
                print(str(scidm_num)+'.',package, now_time)

        #如果不是 ipgod 的資料則印出組織名稱
        else:
            #不是 ipgod 的資料將 id 加入 sqlite 中
            c.execute('insert into patchDone(id) values(?)', (package,))
            conn.commit()
            other_data_num += 1
            print(str(other_data_num)+'.', gov_data['organization']['title'], now_time)
            
    except:
        #不是 ipgod 的資料將 id 加入 sqlite 中
        c.execute('insert into patchDone(id) values(?)', (package,))
        conn.commit()
        other_data_num += 1
        print(str(other_data_num)+'.', package, now_time)

1. 601000000a-000108 I00 publicinformation 10:04:08
1. 高雄市政府資料開放平台 10:04:10
2. 387150000i-003529 I00 publicinformation 10:04:11
3. 376580000a-7e94e8 I00 publicinformation 10:04:12
4. 382000000a-002427 I00 publicinformation 10:04:14
5. a21040000i-000157 100 fertility 10:04:15
6. 380320000a-000114 I00 publicinformation 10:04:18
7. a55000000a-000129 I00 publicinformation 10:04:20
8. 376580000a-97d6f4 I00 publicinformation 10:04:23
9. a27020000g-000033 I00 publicinformation 10:04:25
10. a55000000a-000077 I00 publicinformation 10:04:27
11. 380140000i-000046 B00 medical 10:04:29
12. 376503300j-000023 G00 elderlycare 10:04:31
13. 382000000a-000277 I00 publicinformation 10:04:33
14. a45030000d-000203 800 investment 10:04:34
15. a21040000i-000206 100 fertility 10:04:35
16. a25000000e-000412 I00 publicinformation 10:04:37
17. 504290000f-000013 I00 publicinformation 10:04:39
18. 380130000c-000144 A00 traffic 10:04:41
19. a45030000d-000269 800 investment 10:04:44
20. 387150000i-003500 I00 publicin

155. 395000000a-000032 I00 publicinformation 10:09:20
156. a41000000g-000016 I00 publicinformation 10:09:23
157. a11060000f-000352 I00 publicinformation 10:09:26
158. a27000000g-000030 I00 publicinformation 10:09:28
159. a11040000f-000401 I00 publicinformation 10:09:30
160. a11060000f-000116 I00 publicinformation 10:09:31
161. a59000000n-000016 I00 publicinformation 10:09:33
162. a21010000i-000192 B00 medical 10:09:36
163. 376580000a-dbc139 I00 publicinformation 10:09:38
164. a11060000f-000051 I00 publicinformation 10:09:40
165. a09050000e-000038 I00 publicinformation 10:09:43
166. 382000000a-g00070 I00 publicinformation 10:09:44
167. a62000000g-000037 I00 publicinformation 10:09:46
168. 382000000a-001535 B00 medical 10:09:48
17. 高雄市政府資料開放平台 10:09:51
169. a45030000d-000098 800 investment 10:09:52
18. 臺南市資料開放平台 10:09:53
170. a45000000d-000084 800 investment 10:09:54
171. a51000000a-000127 I00 publicinformation 10:09:55
172. 387200000a-003722 I00 publicinformation 10:09:56
173. 382000000

306. a09000000e-000030 900 travel 10:14:27
307. 700000000a-000014 I00 publicinformation 10:14:28
308. 397000000a-000970 C00 housepurchase 10:14:30
309. a07000000d-000041 I00 publicinformation 10:14:32
310. a62000000g-000108 I00 publicinformation 10:14:33
311. 397000000a-001014 C00 housepurchase 10:14:35
312. a41010000a-000042 I00 publicinformation 10:14:36
313. 380330000e-0000010 I00 publicinformation 10:14:38
34. 高雄市政府資料開放平台 10:14:41
314. 382000000a-g00552 B00 medical 10:14:42
315. a00000000a-000086 I00 publicinformation 10:14:43
316. 379730000a-001648 900 travel 10:14:45
317. a61000000a-000003 D00 election 10:14:46
35. beer 10:14:47
36. 臺南市資料開放平台 10:14:48
318. 380020000a-000020 I00 publicinformation 10:14:48
319. 382000000a-001198 I00 publicinformation 10:14:50
320. a09030000e-000052 300 study 10:14:53
321. a21020000i-000036 E00 lifesafety 10:14:55
322. a09000000e-000226 I00 publicinformation 10:14:56
323. a25000000e-000553 100 fertility 10:14:58
324. 380160000a-000086 C00 housepurch

461. a55010000a-000080 I00 publicinformation 10:18:57
462. a61000000a-000040 D00 election 10:18:59
49. 高雄市政府資料開放平台 10:19:01
463. 382000000a-000618 A00 traffic 10:19:02
464. 701000000a-000089 I00 publicinformation 10:19:05
465. a45030000d-000187 800 investment 10:19:06
466. 376600000a-000223 I00 publicinformation 10:19:07
467. a00000000a-000087 I00 publicinformation 10:19:08
468. 382000000a-001265 I00 publicinformation 10:19:10
469. a27000000g-000034 I00 publicinformation 10:19:12
470. a45010000d-000223 800 investment 10:19:14
50. 高雄市政府資料開放平台 10:19:15
471. 380140000i-000160 E00 lifesafety 10:19:16
472. a57000000a-000444 I00 publicinformation 10:19:18
473. 382000000a-000422 I00 publicinformation 10:19:20
474. 382000000a-g00637 B00 medical 10:19:22
475. 387150000i-003696 I00 publicinformation 10:19:23
476. a25000000e-000445 I00 publicinformation 10:19:24
477. 395000000a-000389 I00 publicinformation 10:19:25
478. 376501521a-bpk3hf I00 publicinformation 10:19:27
479. 397000000a-000061 I00 p

614. 376580000a-63eedf I00 publicinformation 10:23:35
615. 380140000i-000103 E00 lifesafety 10:23:37
616. 387200000a-003158 I00 publicinformation 10:23:38
617. a41000000g-000386 I00 publicinformation 10:23:39
618. a45010000d-000236 800 investment 10:23:41
619. 382000000a-002406 I00 publicinformation 10:23:43
620. 376580000a-50eaf0 I00 publicinformation 10:23:45
621. a21030000i-e3200u I00 publicinformation 10:23:47
622. a45020000d-000452 800 investment 10:23:48
623. a55000100a-000059 900 travel 10:23:50
624. 395000000a-000480 I00 publicinformation 10:23:51
625. a45020000d-000469 800 investment 10:23:52
626. a55000100a-000055 900 travel 10:23:53
627. 382000000a-001522 I00 publicinformation 10:23:55
628. a21010000i-000004 B00 medical 10:23:56
629. 382000000a-001497 I00 publicinformation 10:23:58
630. 380110000g-000015 I00 publicinformation 10:24:00
631. a11010000f-000011 I00 publicinformation 10:24:02
63. 高雄市政府資料開放平台 10:24:03
632. 504390000f-000007 I00 publicinformation 10:24:03
633. a410

...

17015. a53000000a-000117 I00 publicinformation 20:02:11
1852. 臺南市資料開放平台 20:02:13
1853. 高雄市政府資料開放平台 20:02:13
17016. 376600000a-000150 900 travel 20:02:14
1854. 臺南市資料開放平台 20:02:17
1855. 臺南市資料開放平台 20:02:17
1856. 臺南市資料開放平台 20:02:18
17017. 380170000g-000050 I00 publicinformation 20:02:18
17018. 380320000a-000083 I00 publicinformation 20:02:21
17019. a09010000e-000063 I00 publicinformation 20:02:32
17020. a21010000i-000141 B00 medical 20:02:40
17021. 395000000a-000012 I00 publicinformation 20:02:42
17022. 376560000a-1496211082 B00 medical 20:02:44
17023. a57000000a-000772 I00 publicinformation 20:02:47
17024. 387120000j-003010 I00 publicinformation 20:02:48
17025. a25000000e-000290 900 travel 20:02:51
17026. a09050000e-000031 I00 publicinformation 20:02:54
17027. 380150000i-000051 E00 lifesafety 20:02:55
17028. 397000000a-002211 I00 publicinformation 20:03:00
1857. 高雄市政府資料開放平台 20:03:02
17029. 382000000a-000292 A00 traffic 20:03:03
17030. 920090000a-000023 I00 publicinformation 20:03:05
17031

17159. 376580000a-dd1886 I00 publicinformation 20:07:23
17160. a11000003f-000258 E00 lifesafety 20:07:27
17161. 376502000c-32fptu E00 lifesafety 20:07:30
17162. 380140000i-000024 B00 medical 20:07:34
17163. a11080000f-000079 I00 publicinformation 20:07:35
17164. a57000000a-000570 I00 publicinformation 20:07:37
17165. 380120000j-000057 300 study 20:07:38
17166. 376580000a-6d93dd I00 publicinformation 20:07:41
17167. 376501522a-h8g296 I00 publicinformation 20:07:46
17168. a55000100a-000011 I00 publicinformation 20:07:47
17169. 382000000a-001940 C00 housepurchase 20:07:49
17170. 376580000a-921554 I00 publicinformation 20:07:50
17171. a05000000c-057760 B00 medical 20:07:55
17172. 376600000a-000078 I00 publicinformation 20:07:56
17173. a21030000i-e3100g I00 publicinformation 20:07:59
17174. 395000000a-000297 I00 publicinformation 20:08:00
17175. 387140000i-004433 B00 medical 20:08:03
17176. a55000100a-000018 900 travel 20:08:05
17177. 387200000a-003151 I00 publicinformation 20:08:06
17178. 

17303. 382000000a-001689 300 study 20:12:24
17304. 504300000f-000018 I00 publicinformation 20:12:25
17305. a21010000i-000281 B00 medical 20:12:27
17306. 380240000j-000026 I00 publicinformation 20:12:29
17307. 376580000a-197ff0 I00 publicinformation 20:12:32
17308. a09000000e-000186 300 study 20:12:34
17309. a55000000a-000057 600 business 20:12:36
17310. a53000000a-106013 I00 publicinformation 20:12:38
17311. a09000000e-000010 300 study 20:12:41
17312. a09010000e-000002 I00 publicinformation 20:12:46
17313. a11040000f-000043 I00 publicinformation 20:12:47
17314. 382000000a-001666 C00 housepurchase 20:12:50
17315. 376600000a-000246 700 marriage 20:12:52
17316. a25000000e-000272 900 travel 20:12:53
17317. 387170000g-004322 900 travel 20:12:55
17318. a49000000b-000017 I00 publicinformation 20:12:56
17319. a45000000d-000104 I00 publicinformation 20:12:58
17320. a11000003f-000206 E00 lifesafety 20:13:00
17321. a57000000a-001147 I00 publicinformation 20:13:02
1891. 臺南市資料開放平台 20:13:03
17322. a

17453. a51000000a-000091 I00 publicinformation 20:17:18
17454. 397000000a-001302 300 study 20:17:19
17455. 397000000a-000451 I00 publicinformation 20:17:20
17456. 380160000a-000067 I00 publicinformation 20:17:22
17457. a11080000f-000001 I00 publicinformation 20:17:24
17458. 395000000a-000408 I00 publicinformation 20:17:25
17459. a57000000a-000636 I00 publicinformation 20:17:26
17460. 380150000i-000045 I00 publicinformation 20:17:28
17461. 700000000a-000006 I00 publicinformation 20:17:29
17462. a11030000f-000024 I00 publicinformation 20:17:31
17463. 387290000h-004030 A00 traffic 20:17:33
17464. a09030000e-000008 300 study 20:17:35
17465. a11000000f-000069 I00 publicinformation 20:17:37
17466. a62000000g-000142 I00 publicinformation 20:17:38
17467. a11000000f-000301 I00 publicinformation 20:17:39
1900. 臺南市資料開放平台 20:17:40
17468. a05050000c-000136 I00 publicinformation 20:17:40
17469. 376500000a-s09omq I00 publicinformation 20:17:42
17470. 397000000a-002208 I00 publicinformation 20:17:44
1

17601. 376501521a-dapepb I00 publicinformation 20:22:14
17602. 382000000a-002736 I00 publicinformation 20:22:17
17603. a59000000n-000026 I00 publicinformation 20:22:18
17604. a00000000a-000117 I00 publicinformation 20:22:22
17605. 397000000a-000835 I00 publicinformation 20:22:24
17606. 380200000a-000188 500 job 20:22:25
17607. 397000000a-000857 F00 retirement 20:22:29
17608. a09070000e-000021 I00 publicinformation 20:22:30
17609. 387160000a-004750 800 investment 20:22:33
17610. a45030000d-000216 800 investment 20:22:34
17611. a21030000i-e3a00j I00 publicinformation 20:22:36
17612. 504310000f-000037 I00 publicinformation 20:22:38
17613. a11000000f-000522 I00 publicinformation 20:22:39
17614. 380160000a-000049 I00 publicinformation 20:22:40
17615. 382000000a-001063 G00 elderlycare 20:22:42
17616. 382000000a-002079 I00 publicinformation 20:22:43
17617. 382000000a-001708 E00 lifesafety 20:22:44
17618. 380250000g-000017 I00 publicinformation 20:22:46
17619. 387000000a-004211 I00 publicinfor